<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/Extract_WebData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**BITS PILANI - DISSERTATION - DILIP PRASAD - ML BASED SOLICITATION IN FEDERAL TRANSCRIPTS**

Dissertation project for final year


---
This script extracts the queue information where it primarily consists of Urls, we will check if the Url is valid then extract the details

Libraries Used:



*   Beautiful soup
*   Regex
*   UrlLib
*   Regex

Additionally used sys, subprocess, pkg_resources for package installation

Install all the  Packages used in the project if missing

In [ ]:

#Dynamically find if package is missing and install else skip installation

import sys
import subprocess
from traceback import format_exc
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables
!pip3 install urlparse


In [ ]:

import pytz
# it will get the time zone of the specified location
IST = pytz.timezone('Asia/Kolkata')


Intialiaze the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md



In [ ]:

from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
queue_name = "queue-crawledarchiveurls"
queue_service = QueueService(connection_string=connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode



#Fetch the data from web pages via Beautiful soup


In [ ]:

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      print("getSoupObj",url)
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)
  return None

Queue Details with extracted info

In [ ]:

def QueueUrlFound(jsonData):
  print("Queueing data: "+jsonData)
  queue_service.put_message(queue_name,jsonData)

In [ ]:

def ExisitsInArray(arrDet, valToChk):
  try:
    return arrDet.index(valToChk) >= 0
  except: 
    return False
  return False


In [ ]:
allLinks = []
queueMessages = [] #queue messages
def LoopThroughUrls():
  try:
    queueMessages = queue_service.get_messages(queue_name)
  except Exception as e: 
    print('Problem fetching message from queue. Message : "+ str(e)) 
    return None   

  try:   
    for urlFromQueue in queueMessages:
      if urlFromQueue != None:
        url = urlFromQueue.content 
        if ExisitsInArray(allLinks,url) == False): #Check if the Url is not already added to the list
          allLinks.add(url)
          #Queue new data
          txt = getSoupObj(url).text.strip()
          
            
  except Exception as e: 
    print('Problem processing urls. Message : "+ str(e))
    return None


In [ ]:

from datetime import datetime

if __name__ == '__main__':
  AppendLog("initiaing crawling: "+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
  LoopThroughUrls()
  AppendLog("End of crawling: "+ datetime.now(IST).strftime("%d/%m/%Y %H:%M:%S"))